# IASD NLP Project 

#Code generation with language models


---

Authors: 

> Madina Babazhanova

> Jiheng 

> Henrik Abgaryan



In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.4 MB/s eta 0:00:0

In [ ]:
from datasets import load_dataset

ds = load_dataset("codeparrot/github-code", streaming=True, split="train")
print(next(iter(ds)))

#OUTPUT:
{
 'code': "import mod189 from './mod189';\nvar value=mod189+1;\nexport default value;\n",
 'repo_name': 'MirekSz/webpack-es6-ts',
 'path': 'app/mods/mod190.js',
 'language': 'JavaScript',
 'license': 'isc',
 'size': 73
}

{'code': '\'use strict\';\n\nvar clear          = require(\'es5-ext/array/#/clear\')\n  , eIndexOf       = require(\'es5-ext/array/#/e-index-of\')\n  , setPrototypeOf = require(\'es5-ext/object/set-prototype-of\')\n  , callable       = require(\'es5-ext/object/valid-callable\')\n  , d              = require(\'d\')\n  , ee             = require(\'event-emitter\')\n  , Symbol         = require(\'es6-symbol\')\n  , iterator       = require(\'es6-iterator/valid-iterable\')\n  , forOf          = require(\'es6-iterator/for-of\')\n  , Iterator       = require(\'./lib/iterator\')\n  , isNative       = require(\'./is-native-implemented\')\n\n  , call = Function.prototype.call, defineProperty = Object.defineProperty\n  , SetPoly, getValues;\n\nmodule.exports = SetPoly = function (/*iterable*/) {\n\tvar iterable = arguments[0];\n\tif (!(this instanceof SetPoly)) return new SetPoly(iterable);\n\tif (this.__setData__ !== undefined) {\n\t\tthrow new TypeError(this + " cannot be reinitialized");\n\t}

{'code': "import mod189 from './mod189';\nvar value=mod189+1;\nexport default value;\n",
 'repo_name': 'MirekSz/webpack-es6-ts',
 'path': 'app/mods/mod190.js',
 'language': 'JavaScript',
 'license': 'isc',
 'size': 73}

In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("codeparrot/codeparrot")
model = AutoModelWithLMHead.from_pretrained("codeparrot/codeparrot")

inputs = tokenizer("def hello_world():", return_tensors="pt")
outputs = model(**inputs)

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model="codeparrot/codeparrot")
outputs = pipe("def hello_world():")

#3 - Code model evaluation

In [ ]:
from datasets import load_dataset
from evaluate import load

human_eval = load_dataset("openai_humaneval")
code_eval_metric = load("code_eval")

In [ ]:
test_cases = ["assert add(2,3)==5"]
candidates = [["def add(a,b): return a*b", "def add(a, b): return a+b"]]
pass_at_k, results = code_eval_metric.compute(references=test_cases, predictions=candidates, k=[1, 2])
print(pass_at_k)
{'pass@1': 0.5, 'pass@2': 1.0}

In [ ]:
def truncate_number(number: float) -> float:
    """ Given a positive floating point number, it can be decomposed into
    and integer part (largest integer smaller than given number) and decimals
    (leftover part always smaller than 1).

    Return the decimal part of the number.
    >>> truncate_number(3.5)
    0.5
    """

#Full training pipeline

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_dataset
import numpy as np
from tqdm import tqdm


# Load the dataset
ds = load_dataset("codeparrot/github-code", streaming=True, split="train")

# Load the GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Define the training loop
def train_gpt(model, tokenizer, data_loader, num_epochs=10, learning_rate=0.0001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in tqdm(range(num_epochs)):
        for batch in data_loader:
            # Tokenize the batch of code snippets
            inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True)
            labels = inputs["input_ids"].clone()

            # Shift the labels to the right by one token
            labels[:, :-1] = inputs["input_ids"][:, 1:]
            labels[:, -1] = tokenizer.eos_token_id

            # Move the inputs and labels to the GPU
            inputs = {k: v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)

            # Generate code snippets with the model
            outputs = model(**inputs, labels=labels)
            loss = outputs.loss

            # Backpropagation and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

# Define the data loader
class GithubCodeDataset(torch.utils.data.IterableDataset):
    def __init__(self, ds):
        self.ds = ds
    def __iter__(self):
        while True:
            for ex in self.ds:
                yield ex["code"]


In [ ]:
dataset = GithubCodeDataset(ds)
data_loader = DataLoader(dataset, batch_size=4)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
train_gpt(model, tokenizer, data_loader, num_epochs=10)